# Lesson 2: Machine Learning Workflows and DAGs


## Learning objectives of this lesson

* Take ML from prototype to production
* Appreciate where the focus of scientists needs to be in this stack
* Write machine learning flows in Metaflow for 
    - Random forests
    - Boosted trees
    - Neural networks
* Train models in parallel and choose most performant one using Metaflow

This lesson will focus on building local machine learning workflows using Metaflow, which allows data scientists to focus on the top layers of the ML stack, while having access to the infrastructural layers. 

For example, Metaflow helps with orchestration, and it integrates with other popular orchestrators like Argo, Step Functions and soon Airflow, so you don’t have to learn a separate tool just for orchestration.

While there are many other tools just taking care of orchestration, like Airflow, they are not specifically meant for data scientists and require you to spend much more time thinking about infrastructure, like where and how to run the tasks and how to move data between tasks. Metaflow was created at Netflix specifically to serve the day-to-day needs of data scientists, so you don’t have to keep reinventing the wheel.

Attendees will get a feel for writing flows and DAGs to define the steps in their workflows. We’ll also use DAG cards to visualize our ML workflows. This lesson will be local computation and in the next lesson, we’ll burst to the cloud.

In this section, we take the machine learning scripts from the previous lesson and turn them into flows. Currently, in the spirit of not introducing more tools, we'll write our flows in notebook cells and we'll execute them using the command line in Jupyter Lab.

## The Modern Data Stack for Production ML

There are many layers to the modern ML stack. The following is from [Effective Data Science Infrastructure](https://www.manning.com/books/effective-data-science-infrastructure), published by Manning.

![flow0](../img/modern-ML-stack.jpg)

Although they are all incredibly important, data scientists should NOT be expected to be experts at all of them:

![flow0](../img/data-triangle.jpg)

Let's have a look at some of what data scientists do to interact with the infrastructural layers:

![flow0](../img/ds-workflow.jpg)

And let's not forget that this is an iterative process!

![flow0](../img/data-science-iteration.jpg)

## Orchestrating your Machine Learning workflows

The complexity of ML workflows can be become complicated very quickly. To avoid a jungle of data pipelines and models, we need organizational principles for our workflows and _directed acyclic graphs_ have become a standard for this:

<img src="../img/flow_ex_0.png" alt="DAG" style="width: 400px;"/>

DAGs can often be more complicated:

![flow0](../img/flow_ex_1.png)

## A Word on the Basics of Metaflow

> Metaflow follows the dataflow paradigm which models a program as a directed graph of operations. This is a natural paradigm for expressing data processing pipelines, machine learning in particular. We call the graph of operations a flow. You define the operations, called steps, which are nodes of the graph and contain transitions to the next steps, which serve as edges.

> Metaflow sets some constraints on the structure of the graph. For starters, every flow needs a step called start and a step called end. An execution of the flow, which we call a run, starts at start. The run is successful if the final end step finishes successfully. What happens between start and end is up to you.

-- The [Metaflow docs](https://docs.metaflow.org/metaflow/basics).

## Crafting Metaflows for Machine Learning!

ML flows can be broken down into steps, such as:

- importing data
- processing, wrangling, and/or transforming the data
- data validation
- model configuration
- model training, and
- model deployment.

The first flow we write will be a template showing these steps.

As stated above, in the spirit of not introducing more tools, we'll write our flows in notebook cells and we'll execute them using the command line in Jupyter Lab. Having said that, we encourage everybody to get comfortable writing Python scripts and using the command line as much as possible! Notebooks are wonderful for many things but not necessarily for writing and executing ML workflows.


At the end of the day, modern data science applications are real software projects. Applying software engineering principles like structuring code as functions and modules is useful and hence as you get up to speed, you might find a modern IDE like Visual Studio Code quite convenient in writing your workflows. Don’t worry, it is still the same code running in flows as what you would write in notebooks, so there’s not much new to learn.

In [1]:
%%writefile ../flows/local/flow_template.py

"""

Template for writing Metaflows

"""

from metaflow import FlowSpec, step, current, card


class Template_Flow(FlowSpec):
    """
    Template for Metaflows.
    You can choose which steps suit your workflow.
    We have included the following common steps:
    - Start
    - Process data
    - Data validation
    - Model configuration
    - Model training
    - Model deployment
    """
    @card
    @step
    def start(self):
        """
        Start Step for a Flow;
        """
        print("flow name: %s" % current.flow_name)
        print("run id: %s" % current.run_id)
        print("username: %s" % current.username)
        from dotenv import load_dotenv
        load_dotenv('notebooks/my.env')
        import os

        print(os.getenv('ROLE'))

        # Call next step in DAG with self.next(...)
        self.next(self.process_raw_data)

    @step
    def process_raw_data(self):
        """
        Read and process data
        """
        print("In this step, you'll read in and process your data")

        self.next(self.data_validation)

    @step
    def data_validation(self):
        """
        Perform data validation
        """
        print("In this step, you'll write your data validation code")

        self.next(self.get_model_config)

    @step
    def get_model_config(self):
        """
        Configure model + hyperparams
        """
        print("In this step, you'll configure your model + hyperparameters")
        self.next(self.train_model)

    @step
    def train_model(self):
        """
        Train your model
        """
        print("In this step, you'll train your model")

        self.next(self.deploy)

    @step
    def deploy(self):
        """
        Deploy model
        """
        print("In this step, you'll deploy your model")

        self.next(self.end)

    @step
    def end(self):
        """
        DAG is done! Congrats!
        """
        print('DAG ended! Woohoo!')


if __name__ == '__main__':
    Template_Flow()


Overwriting ../flows/local/flow_template.py


You can now run this flow by executing the following from the CLI:

We can also (cheekily) execute from this NB using the handy iPython magic shBang (!):

In [2]:
! python ../flows/local/flow_template.py run

Metaflow 2.5.0 executing Template_Flow for user:hba
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2022-04-07 15:30:34.921 Workflow starting (run-id 1649309434917494):
2022-04-07 15:30:34.928 [1649309434917494/start/1 (pid 2501)] Task is starting.
2022-04-07 15:30:35.514 [1649309434917494/start/1 (pid 2501)] flow name: Template_Flow
2022-04-07 15:30:36.215 [1649309434917494/start/1 (pid 2501)] run id: 1649309434917494
2022-04-07 15:30:36.215 [1649309434917494/start/1 (pid 2501)] username: hba
2022-04-07 15:30:36.217 [1649309434917494/start/1 (pid 2501)] Task finished successfully.
2022-04-07 15:30:36.225 [1649309434917494/process_raw_data/2 (pid 2509)] Task is starting.
2022-04-07 15:30:36.778 [1649309434917494/process_raw_data/2 (pid 2509)] In this step, you'll read in and process your data
2022-04-07 15:30:36.849 [1649309434917494/process_raw_data/2 (pid 2509)] Task finished successfully.
2022-04-07 15:30:36.856 [1649309434917494/data_validat

Success! But what are all these outputs? I'm glad that you asked!

![flow0](../img/mf_output.png)

- **Timestamp** denotes when the line was output.
- The information inside the square brackets identifies a **task**.
- Every Metaflow run gets a unique ID, a **run ID**.
- A run executes the steps in order. The step that is currently being executed is denoted by **step name**.
- A step may spawn multiple tasks which are identified by a **task ID**.
- The combination of a flow name, run ID, step name, and a task ID,uniquely identify a task in your Metaflow environment, amongst all runs of any flows. Here, the flow name is omitted since it is the same for all lines. We call this globally unique identifier a **pathspec**.
- Each task is executed by a separate process in your operating system, identified by a **process ID** aka _pid_. You can use any operating system level monitoring tools such as top to monitor resource consumption of a task based on its process ID.
- After the square bracket comes a **log message** that may be a message output by Metaflow itself, like “Task is starting” in this example, or a line output by your code.

### Metaflow cards

We can use MF cards to visualize aspects of our flow. In this case, there's not much to check out but we **can** see the DAG by using the CLI as follows:

```
python flows/local/flow_template.py card view start
```

## Time to Write a Metaflow Flow: Orchestrating our Random Forests

In this section, we'll turn the random forest from the previous lesson into a flow. Recall our RF code from Lesson 1:



```
#Import scikit-learn dataset library
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

#Load dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

clf_rf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)
scores_rf = cross_val_score(clf_rf, X, y, cv=5)
print(scores_rf)
```

In order to turn this into a (Metaflow) flow, you first need to decide what your steps are going to be. In this case, we’ll have distinct steps to 

* Load the data
* Instantiate the model
* Train the model and perform cross-validation

In general, this involves some design choices on the user’s part and we have some general rules of thumb [here](https://docs.metaflow.org/metaflow/basics).





To then write your flow, you

* Import `FlowSpec` and `step` outside your flow
* Include step-specific imports within each step
* Assign any data structures you wish to pass between steps to self





## A Reminder on Why We're Doing This

As there is significant overhead in doing this work, let's be explicit as to the benifits:
- Notebooks are great when you want to tell a concise linear narrative as a single article. That’s why they are so great for teaching and learning. However, real-world applications tend to be non-linear, so it is better to structure the project as an explicit DAG which makes it more understandable for humans and more easily executable by computers.
- A superpower of DAGs is that if you have parallel branches in your DAG, computer can execute them in parallel automatically, making your project finish much faster without you having to change much anything.
- This means that you can e.g. train 200 models - one for each country of the world in parallel, even using multiple computers - which would be impossible to do in a notebook.
- It is much easier to take a workflow, which is a proper Python project, to production than a notebook, in particular when using a project like Metaflow that integrates with modern workflow schedulers and compute platforms like Kubernetes. This will make engineers happy.

In [2]:
%%writefile ../flows/local/rf_flow.py

from metaflow import FlowSpec, step, card

class ClassificationFlow(FlowSpec):
    """
    train a random forest
    """
    @card 
    @step
    def start(self):
        """
        Load the data
        """
        #Import scikit-learn dataset library
        from sklearn import datasets

        #Load dataset
        self.iris = datasets.load_iris()
        self.X = self.iris['data']
        self.y = self.iris['target']
        self.next(self.rf_model)
        

    @step
    def rf_model(self):
        """
        build random forest model
        """
        from sklearn.ensemble import RandomForestClassifier
        
        self.clf = RandomForestClassifier(n_estimators=10, max_depth=None,
            min_samples_split=2, random_state=0)
        self.next(self.train)

        
        
    @step
    def train(self):
        """
        Train the model
        """
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.clf, self.X, self.y, cv=5)
        self.next(self.end)
        
        
    @step
    def end(self):
        """
        End of flow!
        """
        print("ClassificationFlow is all done.")


if __name__ == "__main__":
    ClassificationFlow()

Overwriting ../flows/local/rf_flow.py


Execute the above from the command line with

```bash
python flows/local/rf_flow.py run
```

We can check out the Metaflow card:

```
python flows/local/rf_flow.py card view start
```

## The Unreasonable Effective of Branching Flows

Often, you'll want to train several independent models and then choose the best one. As they're indepedendent, you can train them in parallel. For this, you can use the concept of branching, which is exemplified in this figure:


<img src="../img/flow_ex_0.png" alt="DAG" style="width: 400px;"/>

We'll show how to do this in Metaflow by writing a flow that has random forests, decision trees, and extra trees classifiers, trains them all and chooses the best one.

In [3]:
%%writefile ../flows/local/tree_branch_flow.py

from metaflow import FlowSpec, step, card

class ClassificationFlow(FlowSpec):
    """
    train multiple tree based methods
    """
    @card 
    @step
    def start(self):
        """
        Load the data
        """
        #Import scikit-learn dataset library
        from sklearn import datasets

        #Load dataset
        self.iris = datasets.load_iris()
        self.X = self.iris['data']
        self.y = self.iris['target']
        self.next(self.rf_model, self.xt_model, self.dt_model)
    
                
    @step
    def rf_model(self):
        """
        build random forest model
        """
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.model_selection import cross_val_score
        
        self.clf = RandomForestClassifier(n_estimators=10, max_depth=None,
            min_samples_split=2, random_state=0)
        self.scores = cross_val_score(self.clf, self.X, self.y, cv=5)
        self.next(self.choose_model)

    @step
    def xt_model(self):
        """
        build extra trees classifier
        """
        from sklearn.ensemble import ExtraTreesClassifier
        from sklearn.model_selection import cross_val_score
        

        self.clf = ExtraTreesClassifier(n_estimators=10, max_depth=None,
            min_samples_split=2, random_state=0)

        self.scores = cross_val_score(self.clf, self.X, self.y, cv=5)
        self.next(self.choose_model)

    @step
    def dt_model(self):
        """
        build decision tree classifier
        """
        from sklearn.tree import DecisionTreeClassifier
        from sklearn.model_selection import cross_val_score
        
        self.clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,
            random_state=0)

        self.scores = cross_val_score(self.clf, self.X, self.y, cv=5)

        self.next(self.choose_model)
                        
    @step
    def choose_model(self, inputs):
        """
        find 'best' model
        """
        import numpy as np

        def score(inp):
            return inp.clf,\
                   np.mean(inp.scores)

            
        self.results = sorted(map(score, inputs), key=lambda x: -x[1]) 
        self.model = self.results[0][0]
        self.next(self.end)
        
    @step
    def end(self):
        """
        End of flow!
        """
        print('Scores:')
        print('\n'.join('%s %f' % res for res in self.results))
        print('Best model:')
        print(self.model)


if __name__ == "__main__":
    ClassificationFlow()

Overwriting ../flows/local/tree_branch_flow.py


Execute the above from the command line with

```bash
python flows/local/tree_branch_flow.py run
```

We can also view the Metaflow card:

```
python flows/local/tree_branch_flow.py card view start
```

## Boosted Trees Flow

In this section, we'll turn the XGBoost example from Lesson 1 into a flow:

In [5]:
%%writefile ../flows/local/boosted_flow.py

from metaflow import FlowSpec, step, card


class BSTFlow(FlowSpec):
    """
    train a boosted tree
    """
    @card
    @step
    def start(self):
        """
        Load the data & train model
        """
        import xgboost as xgb
        dtrain = xgb.DMatrix('data/agaricus.txt.train')
        param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic','eval_metric':'logloss'}
        num_round = 2
        bst = xgb.train(param, dtrain, num_round)
        bst.save_model("model.json")
        self.next(self.predict)
        
    @step
    def predict(self):
        """
        make predictions
        """
        import xgboost as xgb

        dtest = xgb.DMatrix('data/agaricus.txt.test')
        # make prediction
        bst = xgb.Booster()
        bst.load_model("model.json")
        preds = bst.predict(dtest)
        self.next(self.end)
        
        
    @step
    def end(self):
        """
        End of flow!
        """
        print("BSTFlow is all done.")


if __name__ == "__main__":
    BSTFlow()

Overwriting ../flows/local/boosted_flow.py


Execute the above from the command line with

```bash
python flows/local/boosted_flow.py run
```

## Your First Deep Learning Metaflow!

In this section, we'll turn the deep learning example above into a flow.

In [6]:
%%writefile ../flows/local/NN_flow.py

from metaflow import FlowSpec, step, Parameter, JSONType, IncludeFile, card
from taxi_modules import init, MODELS, MODEL_LIBRARIES
import json


class NNFlow(FlowSpec):
    """
    train a NN
    """
    @card
    @step
    def start(self):
        """
        Load the data
        """
        from tensorflow import keras

        # the data, split between train and test sets
        (self.x_train, self.y_train), (self.x_test, self.y_test) = keras.datasets.mnist.load_data()
        self.next(self.wrangle)
        
    @step
    def wrangle(self):
        """
        massage data
        """
        import numpy as np
        from tensorflow import keras
        # Model / data parameters
        self.num_classes = 10
        self.input_shape = (28, 28, 1)

        # Scale images to the [0, 1] range
        self.x_train = self.x_train.astype("float32") / 255
        self.x_test = self.x_test.astype("float32") / 255
        # Make sure images have shape (28, 28, 1)
        self.x_train = np.expand_dims(self.x_train, -1)
        self.x_test = np.expand_dims(self.x_test, -1)

        # convert class vectors to binary class matrices
        self.y_train = keras.utils.to_categorical(self.y_train, self.num_classes)
        self.y_test = keras.utils.to_categorical(self.y_test, self.num_classes)
        
        self.next(self.build_model)


    @step
    def build_model(self):
        """
        build NN model
        """
        import tempfile
        import numpy as np
        import tensorflow as tf
        from tensorflow import keras
        from tensorflow.keras import layers

        model = keras.Sequential(
            [
                keras.Input(shape=self.input_shape),
                layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Flatten(),
                layers.Dropout(0.5),
                layers.Dense(self.num_classes, activation="softmax"),
            ]
        )
        model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        with tempfile.NamedTemporaryFile() as f:
            tf.keras.models.save_model(model, f.name, save_format='h5')
            self.model = f.read()
        self.next(self.train)

        
        
    @step
    def train(self):
        """
        Train the model
        """
        import tempfile
        import tensorflow as tf
        self.batch_size = 128
        self.epochs = 15
        
        with tempfile.NamedTemporaryFile() as f:
            f.write(self.model)
            f.flush()
            model =  tf.keras.models.load_model(f.name)
        model.fit(self.x_train, self.y_train, batch_size=self.batch_size, epochs=self.epochs, validation_split=0.1)
        
        self.next(self.end)
        
        
    @step
    def end(self):
        """
        End of flow!
        """
        print("NNFlow is all done.")


if __name__ == "__main__":
    NNFlow()

Overwriting ../flows/local/NN_flow.py


Execute the above from the command line with

```bash
python flows/local/NN_flow.py run
```

We can also view the Metaflow card:

```
python flows/local/NN_flow.py card view start
```

## Lesson Recap

In this lesson, we covered the following:

* Understand what the Modern Data Stack for Production ML looks like
* Appreciate where the focus of scientists needs to be in this stack
* Write machine learning flows in Metaflow for 
    - Random forests
    - Boosted trees
    - Neural networks
* Train models in parallel and choose most performant one using Metaflow

In the next lesson, we'll take these machine learning workflows and see how to get them running on the cloud!